# 1. Optimal trading strategy and supply/demand dynamics

The paper "Optimal trading strategy and supply/demand dynamics" studies how the intertemporal supply and demand dynamics of a security affect the optimal trading strategy. It develops a general framework for a limit order book (LOB) market to capture these dynamics and demonstrates that the optimal strategy depends on the dynamic properties of supply and demand, particularly the market's resilience, which is the speed at which supply and demand recover to their steady state after a trade.

## Mathematical Model

### Basic Framework
1. **Limit Order Book (LOB)**:
   - The LOB represents the supply and demand of a security via limit orders posted in an electronic trading system.
   - The model focuses on how the LOB evolves over time, capturing the dynamics a large trader faces.

2. **Key Features**:
   - **Bid-Ask Spread**: The difference between the highest bid price and the lowest ask price.
   - **Market Depth**: The volume of orders at different price levels.
   - **Resilience**: The rate at which the LOB replenishes after being impacted by trades.

### Dynamic Supply/Demand Model
1. **Initial Setup**:
   - The fundamental value of the security, $F_t$, follows a Brownian motion: $F_t = F_0 + \sigma Z_t$, where $Z_t$ is a standard Brownian motion.

2. **LOB Shape**:
   - The LOB at time $t$ is described by the density of sell orders $q_t(P)$ at price $P$.

3. **Trade Impact**:
   - A trade of size $x_t$ at time $t$ shifts the ask price $A_t$:
     $$
     A_{t+} = V_t + \frac{x_t}{q}
     $$
   - The mid-quote price $V_t$ shifts permanently due to the trade:
     $$
     V_{t+} = F_t + \lambda x_t
     $$
   - The ask price $A_t$ dynamically evolves as new limit orders arrive:
     $$
     A_t = V_t + \frac{x_t}{q} e^{-rt}
     $$

4. **Optimal Strategy**:
   - The goal is to minimize the expected total cost of executing an order $X_0$ over a time period $[0, T]$.
   - The value function $J_t$ represents the expected cost of future trades given the state variables $X_t$ (remaining shares to buy), $A_t$ (current ask price), and $V_t$ (mid-quote price).

5. **Bellman Equation**:
   - The Bellman equation for the optimal execution strategy is:
     $$
     J_t = \min_{x_t} \mathbb{E} \left[ \int_t^T A_s x_s \, ds + \sum_{t \in T} A_t x_t \right]
     $$
   - The value function $J_t$ is given by:
     $$
     J_t = (F_t + \frac{s}{2}) X_t + \lambda X_0 X_t + a_t X_t^2 + b_t D_t X_t + g_t D_t^2
     $$




### Discrete-Time Solution

Suppose that trading times are fixed at $t_n = n\Delta t$, where $\Delta t = \frac{T}{N}$ and $n = 0, 1, \ldots, N$. We consider the corresponding strategies $x_{[0,T]} = \{x_0, x_1, \ldots, x_n\}$ within the strategy set $Y_D$ defined in Section 2. Using the dynamics of the state variables, the optimal execution problem can be formulated as:

$$
J_0 = \min_{\{x_0, \ldots, x_N\}} \mathbb{E}_0 \left[ \sum_{n=0}^N \left( A_{t_n} + \frac{x_n}{2q} \right) x_n \right]
$$

subject to:

$$
A_{t_n} = F_{t_n} + \lambda (X_0 - X_{t_n}) + \frac{s}{2} + \sum_{i=0}^{n-1} x_i k e^{-r(t_n - t_i)}
$$

where $F_t$ follows a random walk.

The solution to the optimal execution problem is given by:

$$
x_n = -\frac{1}{2} d_{n+1} \left[ D_{t_n} (1 - b_{n+1} e^{-r \Delta t} + 2k g_{n+1} e^{-2r \Delta t}) - X_{t_n} (\lambda + 2 a_{n+1} - b_{n+1} k e^{-r \Delta t}) \right]
$$

with $x_N = X_N$ and $D_t = A_t - V_t - \frac{s}{2}$. The expected cost for future trades under the optimal strategy is determined according to:

$$
J_{t_n} = (F_{t_n} + \frac{s}{2}) X_{t_n} + \lambda X_0 X_{t_n} + a_n X_{t_n}^2 + b_n D_{t_n} X_{t_n} + g_n D_{t_n}^2
$$

where the coefficients $a_{n+1}, b_{n+1}, g_{n+1}$, and $d_{n+1}$ are determined recursively as follows:

$$
a_n = a_{n+1} - \frac{1}{4} d_{n+1} (\lambda + 2 a_{n+1} - b_{n+1} k e^{-r \Delta t})^2
$$

$$
b_n = b_{n+1} e^{-r \Delta t} + \frac{1}{2} d_{n+1} (1 - b_{n+1} e^{-r \Delta t} + 2 k g_{n+1} e^{-2r \Delta t}) (\lambda + 2 a_{n+1} - b_{n+1} k e^{-r \Delta t})
$$

$$
g_n = g_{n+1} e^{-2r \Delta t} - \frac{1}{4} d_{n+1} (1 - b_{n+1} e^{-r \Delta t} + 2 g_{n+1} k e^{-2r \Delta t})^2
$$

with

$$
d_{n+1} = \left[ \frac{1}{2q} + a_{n+1} - b_{n+1} k e^{-r \Delta t} + g_{n+1} k^2 e^{-2r \Delta t} \right]^{-1}
$$

and terminal conditions:

$$
a_N = \frac{1}{2q} - \lambda, \quad b_N = 1, \quad g_N = 0
$$


In [3]:
import numpy as np

def optimal_execution_strategy(N, q, r, lambda_, T, s, F_t, X_0):
    """
    Calculate the optimal execution strategy for given parameters.

    Parameters:
    N (int): Number of trading intervals.
    q (float): Market depth.
    r (float): Resilience parameter.
    lambda_ (float): Permanent price impact coefficient.
    T (float): Trading horizon.
    s (float): Bid-ask spread.
    F_t (float): Fundamental value of the security at time t.
    X_0 (float): Initial order size.

    Returns:
    x (numpy array): Optimal trade sizes at each interval.
    A (numpy array): Ask prices at each interval.
    V (numpy array): Mid-quote prices at each interval.
    J (numpy array): Expected costs at each interval.
    """

    delta_t = T / N
    a = np.zeros(N + 1)
    b = np.zeros(N + 1)
    g = np.zeros(N + 1)
    d = np.zeros(N + 1)
    x = np.zeros(N + 1)
    A = np.zeros(N + 1)
    V = np.zeros(N + 1)
    J = np.zeros(N + 1)
    
    # Terminal conditions
    a[N] = 1 / (2 * q) - lambda_
    b[N] = 1
    g[N] = 0
    
    for n in reversed(range(N)):
        d[n + 1] = 1 / (1 / (2 * q) + a[n + 1] - b[n + 1] * np.exp(-r * delta_t) + g[n + 1] * (np.exp(-2 * r * delta_t)))
        a[n] = a[n + 1] - 1 / 4 * d[n + 1] * (lambda_ + 2 * a[n + 1] - b[n + 1] * np.exp(-r * delta_t))**2
        b[n] = b[n + 1] * np.exp(-r * delta_t) + 1 / 2 * d[n + 1] * (1 - b[n + 1] * np.exp(-r * delta_t) + 2 * g[n + 1] * np.exp(-2 * r * delta_t)) * (lambda_ + 2 * a[n + 1] - b[n + 1] * np.exp(-r * delta_t))
        g[n] = g[n + 1] * np.exp(-2 * r * delta_t) - 1 / 4 * d[n + 1] * (1 - b[n + 1] * np.exp(-r * delta_t) + 2 * g[n + 1] * np.exp(-2 * r * delta_t))**2

    D_t = A - V - s / 2
    for n in range(N):
        x[n] = -1 / 2 * d[n + 1] * (D_t[n] * (1 - b[n + 1] * np.exp(-r * delta_t) + 2 * g[n + 1] * np.exp(-2 * r * delta_t)) - X_0 * (lambda_ + 2 * a[n + 1] - b[n + 1] * np.exp(-r * delta_t)))
    
    x[N] = X_0
    for n in range(N + 1):
        J[n] = (F_t + s / 2) * X_0 + lambda_ * X_0 * X_0 + a[n] * X_0**2 + b[n] * D_t[n] * X_0 + g[n] * D_t[n]**2
    
    return x, A, V, J

# Example usage
N = 10
q = 5000
r = 0.1
lambda_ = 0.01
T = 1
s = 0.02
F_t = 100
X_0 = 100000

x, A, V, J = optimal_execution_strategy(N, q, r, lambda_, T, s, F_t, X_0)

print("Optimal trade sizes at each interval:", x)
print("Ask prices at each interval:", A)
print("Mid-quote prices at each interval:", V)
print("Expected costs at each interval:", J)


Optimal trade sizes at each interval: [  7742.534132     8817.58944249  10092.03729572  11641.81448769
  13585.97880261  16122.669833    19608.2162923   24754.89727704
  33224.21781396  49999.99995024 100000.        ]
Ask prices at each interval: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Mid-quote prices at each interval: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Expected costs at each interval: [4.42742008e+09 4.39663421e+09 4.35592059e+09 4.30144947e+09
 4.22721354e+09 4.12321230e+09 3.97145793e+09 3.73599038e+09
 3.33292289e+09 2.51062459e+09 1.10000000e+07]


# 2. Optimal Liquidity Trading
The paper investigated the optimal trading strategies for liquidity traders who aim to trade a fixed number of shares within a certain time horizon while minimizing the mean and variance of trading costs. The study provides explicit formulas for optimal trading strategies, showing that risk-averse liquidity traders reduce their order sizes over time and execute a higher fraction of their total trading volume in early periods when price volatility or liquidity increases.

## Key Findings

1. **Risk Aversion and Trading Strategy**:
   - Risk-averse traders tend to reduce their order sizes over time.
   - Traders execute a higher fraction of their total volume in early periods when price volatility or liquidity increases.

2. **Impact of Transaction Fees**:
   - Higher transaction fees lead to less frequent trading.
   - Traders adjust their strategies based on the trade-off between price impact and opportunity costs.

3. **Multi-Asset Trading**:
   - The model extends to multi-asset cases, showing that price effects across assets significantly impact trading behavior.
   - Cross-asset price impacts and correlations influence the optimal trading strategy.

## Mathematical Model

### Basic Assumptions
- A single asset is traded over $N$ periods.
- Traders submit their orders simultaneously, and the price change from one period to the next depends on the aggregate excess demand.
- Orders are placed before the price change is known, and only market orders are considered.

### Price Dynamics
- The initial price of the asset at time $n$, $\hat{p}_n$, observed before choosing the quantity $q_n$, is the last price update after trades in the previous period $n-1$.
- The transaction price $p_n$ is given by:
  $$
  p_n = \hat{p}_n + \lambda_n (q_n + \eta_n)
  $$
  where $\eta_n$ is a random variable representing the unknown volume of other traders.

### Price Update Rule
- After all trades at time $n$, the new price update for the next period is:
  $$
  \hat{p}_{n+1} = \alpha \hat{p}_n + (1 - \alpha)p_n + \epsilon_{n+1}
  $$
  where $0 \leq \alpha \leq 1$ and $\epsilon_{n+1}$ incorporates news into the price.

### Optimization Problem
- The liquidity trader's optimization problem aims to minimize the mean and variance of the total execution costs:
  $$
  L(Q,N) = \inf_{\{q_n\}} \mathbb{E} \left[\sum_{n=1}^N p_n q_n\right] + \frac{R}{2} \text{Var} \left[\sum_{n=1}^N p_n q_n\right]
  $$
  subject to:
  $$
  \sum_{n=1}^N q_n = Q
  $$
  where $Q$ is the total number of shares to be traded, and $R$ is the risk-aversion coefficient.


#### Optimal Trading Sequence
- The optimal trading sequence $\{q_n\}$ can be derived using dynamic programming and is given by:
  $$
  q_n = \theta_n Q_n
  $$
- For $1 \leq n \leq N$, where:
  $$
  \theta_n = \frac{1 - \lambda_n (1 + \alpha + R \alpha \lambda_n \sigma^2_{\eta})}{2 \mu_{n+1} + R (\alpha^2 \lambda_n^2 \sigma^2_{\eta} + \sigma^2_{\epsilon})}
  $$

#### Cost Function Evolution
- The minimal costs evolve according to:
  $$
  L_n(\hat{p}_{n-1}, Q_n) = [\hat{p}_{n-1} - \alpha \lambda_{n-1} (Q_{n-1} + \eta_{n-1})] Q_n + \mu_n Q_n^2
  $$

#### Recursive Equation for $\mu_n$
- The coefficient $\mu_n$ satisfies the recursive equation:
  $$
  \mu_n = \alpha \lambda_{n-1} + \lambda_n - \frac{(1 + \alpha) \lambda_n^2}{2 \mu_{n+1} + R (\alpha^2 \lambda_n^2 \sigma^2_{\eta} + \sigma^2_{\epsilon})}
  $$
- With the boundary condition:
  $$
  \mu_N = \alpha \lambda_{N-1} + \lambda_N (1 + \frac{R}{2} \lambda_N \sigma^2_{\eta})
  $$
